In [5]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda:0

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [9]:
import time
zeitanfang = time.time()

In [ ]:
import pandas as pd
df = pd.read_csv("Exampledataframe.csv", sep=';', header=1)
df.drop(index=df.index[0], 
        axis=0, 
        inplace=True)
df = df.rename(columns={'answer_nps_comment': 'Review'})
df.head()

# Pre Processing

In [11]:
df.drop(['interview_id', 'answer_nps'], axis=1, inplace=True)

In [ ]:
#Create List of reviews
train_text_list = df['Review'].tolist()
print(train_text_list)

In [ ]:
#Tokenize the sentences
import nltk
from nltk import word_tokenize
from nltk import StanfordTagger
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

tagged_list = []
for i in train_text_list:
    tagged_text_list = nltk.word_tokenize(i)
    pos_tagged = nltk.pos_tag(tagged_text_list)
    tagged_list.append(pos_tagged)


print(tagged_list)

In [ ]:
#Filter the word with tag- noun,adjective,verb,adverb, number
def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['CD','NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list

#train list after filter
final_tagged_list=filterTag(tagged_list)
print(final_tagged_list)

In [ ]:
data={'Review':final_tagged_list}
data = pd.DataFrame(data)
data.index = range(1,len(data)+1)
    
df["Review"] = data["Review"]
df.head(5)

# Training Aspect Extraction model

In [17]:
from sklearn.model_selection import train_test_split


X= df.Review
y = df.drop('Review',1)

y = y.fillna(10)
y = y.astype(str).astype(int)
y[y <= 0] = 1
y[y == 10] = 0

y["comment_text"] = X
df = y

/tmp/ipykernel_2333817/1065055103.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  y = df.drop('Review',1)


In [18]:
!pip3 install pytorch_lightning
!pip3 install transformers
!pip3 install seaborn

In [19]:
import pandas as pd

import numpy as np

from tqdm.auto import tqdm

import torch

import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from torchmetrics.functional import accuracy, f1, auroc

from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns

from pylab import rcParams

import matplotlib.pyplot as plt

from matplotlib import rc

%matplotlib inline

%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [20]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

#Only for Bagging
#train_df = train_df.sample(frac=1, replace=True, random_state=50)


train_df.shape, val_df.shape

((1251, 15), (139, 15))

In [22]:
from transformers import BertTokenizer, BertForMaskedLM
#BERT_MODEL_NAME = 'bert-base-cased'

BERT_MODEL_NAME = 'bert-large-cased'

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [25]:
class ToxicCommentsDataset(Dataset):

  def __init__(

    self,

    data: pd.DataFrame,

    tokenizer: BertTokenizer,

    max_token_len: int = 128

  ):

    self.tokenizer = tokenizer

    self.data = data

    self.max_token_len = max_token_len

  def __len__(self):

    return len(self.data)

  def __getitem__(self, index: int):

    data_row = self.data.iloc[index]

    comment_text = data_row.comment_text

    labels = data_row[LABEL_COLUMNS]

    encoding = self.tokenizer.encode_plus(

      comment_text,

      add_special_tokens=True,

      max_length=self.max_token_len,

      return_token_type_ids=False,

      padding="max_length",

      truncation=True,

      return_attention_mask=True,

      return_tensors='pt',

    )

    return dict(

      comment_text=comment_text,

      input_ids=encoding["input_ids"].flatten(),

      attention_mask=encoding["attention_mask"].flatten(),

      labels=torch.FloatTensor(labels)

    )

In [26]:
MAX_TOKEN_COUNT = 512
train_dataset = ToxicCommentsDataset(

  train_df,

  tokenizer,

  max_token_len=MAX_TOKEN_COUNT

)

sample_item = train_dataset[0]

sample_item.keys()

dict_keys(['comment_text', 'input_ids', 'attention_mask', 'labels'])

In [27]:
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)

sample_batch = next(iter(DataLoader(train_dataset, batch_size=32, num_workers=2)))

sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(torch.Size([32, 512]), torch.Size([32, 512]))

In [28]:
bert_model.config.hidden_size

1024

In [29]:
class ToxicCommentDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=1, max_token_len=128):

    super().__init__()

    self.batch_size = batch_size

    self.train_df = train_df

    self.test_df = test_df

    self.tokenizer = tokenizer

    self.max_token_len = max_token_len

  def setup(self, stage=None):

    self.train_dataset = ToxicCommentsDataset(

      self.train_df,

      self.tokenizer,

      self.max_token_len

    )

    self.test_dataset = ToxicCommentsDataset(

      self.test_df,

      self.tokenizer,

      self.max_token_len

    )

  def train_dataloader(self):

    return DataLoader(

      self.train_dataset,

      batch_size=self.batch_size,

      shuffle=True,

      num_workers=32

    )

  def val_dataloader(self):

    return DataLoader(

      self.test_dataset,

      batch_size= 1,

      num_workers=32

    )

  def test_dataloader(self):

    return DataLoader(

      self.test_dataset,

      batch_size= 1,

      num_workers=32

    )

In [30]:
N_EPOCHS = 100

BATCH_SIZE = 1

LEARNING_RATE = 2e-5

data_module = ToxicCommentDataModule(

  train_df,

  val_df,

  tokenizer,

  batch_size=BATCH_SIZE,

  max_token_len=MAX_TOKEN_COUNT

)

In [31]:
class ToxicCommentTagger(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):

    super().__init__()

    self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)

    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    self.n_training_steps = n_training_steps

    self.n_warmup_steps = n_warmup_steps

    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):

    output = self.bert(input_ids, attention_mask=attention_mask)

    output = self.classifier(output.pooler_output)

    output = torch.sigmoid(output)

    loss = 0

    if labels is not None:

        loss = self.criterion(output, labels)

    return loss, output

  def training_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]

    attention_mask = batch["attention_mask"]

    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels)

    self.log("train_loss", loss, prog_bar=True, logger=True)

    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]

    attention_mask = batch["attention_mask"]

    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels)

    self.log("val_loss", loss, prog_bar=True, logger=True)

    return loss

  def test_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]

    attention_mask = batch["attention_mask"]

    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels)

    self.log("test_loss", loss, prog_bar=True, logger=True)

    return loss

  def training_epoch_end(self, outputs):

    labels = []

    predictions = []

    for output in outputs:

      for out_labels in output["labels"].detach().cpu():

        labels.append(out_labels)

      for out_predictions in output["predictions"].detach().cpu():

        predictions.append(out_predictions)

    labels = torch.stack(labels).int()

    predictions = torch.stack(predictions)

    for i, name in enumerate(LABEL_COLUMNS):

      class_roc_auc = auroc(predictions[:, i], labels[:, i])

      self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=LEARNING_RATE)

    scheduler = get_linear_schedule_with_warmup(

      optimizer,

      num_warmup_steps=self.n_warmup_steps,

      num_training_steps=self.n_training_steps

    )

    return dict(

      optimizer=optimizer,

      lr_scheduler=dict(

        scheduler=scheduler,

        interval='step'

      )

    )

In [32]:
steps_per_epoch=len(train_df) // BATCH_SIZE

total_training_steps = steps_per_epoch * N_EPOCHS

In [33]:
warmup_steps = total_training_steps // 5

warmup_steps, total_training_steps

(10008, 50040)

In [34]:
model = ToxicCommentTagger(

  n_classes=len(LABEL_COLUMNS),

  n_warmup_steps=warmup_steps,

  n_training_steps=total_training_steps

)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
checkpoint_callback = ModelCheckpoint(

  dirpath="checkpoints",

  filename="best-checkpoint",

  save_top_k=1,

  verbose=True,

  monitor="val_loss",

  mode="min"

)

In [36]:
logger = TensorBoardLogger("lightning_logs", name="toxic-comments")

In [37]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

In [38]:
trainer = pl.Trainer(

  logger=logger,

  checkpoint_callback=checkpoint_callback,

  callbacks=[early_stopping_callback],

  max_epochs=N_EPOCHS,

  gpus=1,

  progress_bar_refresh_rate=30

)

/home/moritz/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f9bc1eabdf0>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f9bc1eabdf0>)`.
  rank_zero_deprecation(
/home/moritz/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: Tr

In [ ]:
trainer.fit(model, data_module)

In [ ]:
trained_model = ToxicCommentTagger.load_from_checkpoint(

  trainer.checkpoint_callback.best_model_path,

  n_classes=len(LABEL_COLUMNS)

)

In [41]:
trained_model = trained_model.to(device)

val_dataset = ToxicCommentsDataset(

  val_df,

  tokenizer,

  max_token_len=MAX_TOKEN_COUNT

)

predictions = []

labels = []

for item in tqdm(val_dataset):
    
  _, prediction = trained_model(

    item["input_ids"].unsqueeze(dim=0).to(device),

    item["attention_mask"].unsqueeze(dim=0).to(device)

  )

  predictions.append(prediction.detach().cpu().flatten())

  labels.append(item["labels"].detach().cpu().int())

predictions = torch.stack(predictions).detach().cpu()

labels = torch.stack(labels).detach().cpu()

  0%|          | 0/139 [00:00<?, ?it/s]

In [42]:
THRESHOLD = 0.5
accuracy(predictions, labels, threshold=THRESHOLD)
print(predictions, labels)

tensor([[9.3341e-01, 9.7961e-01, 7.9026e-03,  ..., 1.0088e-03, 1.2697e-04,
         8.8558e-05],
        [9.8927e-01, 1.0888e-02, 6.0330e-03,  ..., 3.4633e-01, 4.0538e-04,
         4.9690e-04],
        [4.5959e-02, 9.9311e-01, 6.5355e-03,  ..., 1.8537e-03, 9.0896e-05,
         9.9260e-05],
        ...,
        [3.7834e-01, 9.6892e-01, 2.4645e-02,  ..., 6.4104e-03, 1.4218e-03,
         1.5766e-03],
        [9.8042e-01, 3.1360e-04, 9.0004e-04,  ..., 2.9400e-02, 6.4657e-05,
         7.7782e-05],
        [9.9287e-01, 1.6291e-03, 8.1279e-04,  ..., 1.0137e-02, 1.6302e-05,
         1.9549e-05]]) tensor([[1, 1, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 0, 0, 0],
        ...,
        [1, 1, 0,  ..., 1, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)


In [45]:
predictions2 = predictions.tolist()
data2={'BERT_Predictions':predictions2}
data2 = pd.DataFrame(data2)

In [46]:
data2.head(5)

,BERT_Predictions
0,"[0.9334074258804321, 0.9796074628829956, 0.007..."
1,"[0.9892672300338745, 0.010887877084314823, 0.0..."
2,"[0.04595877602696419, 0.9931136965751648, 0.00..."
3,"[0.02699887938797474, 0.9936147928237915, 0.00..."
4,"[0.6075873970985413, 0.9719584584236145, 0.004..."


In [47]:
data2.to_csv('ac/bert_predictions.csv', index=False)  
#data2.to_csv('bagging/ac/bert_predictions.csv', index=False)  

In [ ]:
zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

# Training Aspect-based Sentiment Analysis model

In [136]:
#Create binary datasets for positive, negative and neutral sentiments
from sklearn.model_selection import train_test_split

X = df.Review
y = df.drop('Review',1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


#Positive Dataset
#Train Dataset
df_positive_train_X = X_train
df_positive_train_y = y_train

df_positive_train_y = df_positive_train_y.fillna(0)
df_positive_train_y = df_positive_train_y.astype(str).astype(int)
df_positive_train_y[df_positive_train_y <= 0] = 0

df_positive_train = df_positive_train_X.to_frame().merge(df_positive_train_y, how='inner', left_index=True, right_index=True)

df_positive_train = df_positive_train.sample(frac=2, replace=True, random_state=50)


#Test Dataset
df_positive_test_X = X_test
df_positive_test_y = y_test

df_positive_test_y = df_positive_test_y.fillna(0)
df_positive_test_y = df_positive_test_y.astype(str).astype(int)
df_positive_test_y[df_positive_test_y <= 0] = 0

df_positive_test = df_positive_test_X.to_frame().merge(df_positive_test_y, how='inner', left_index=True, right_index=True)

/tmp/ipykernel_2776414/2761705871.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  y = df.drop('Review',1)


In [ ]:
df_positive_train.rename(columns={"Review": "comment_text"})

In [ ]:
LABEL_COLUMNS = df_positive_train.columns.tolist()[1:15]
print(LABEL_COLUMNS)

In [139]:
df_positive_train.shape

(2502, 15)

In [140]:
from transformers import BertTokenizer, BertForMaskedLM
#BERT_MODEL_NAME = 'bert-base-cased'

BERT_MODEL_NAME = 'bert-large-cased'

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

In [141]:
import pandas as pd

import numpy as np

from tqdm.auto import tqdm

import torch

import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from torchmetrics.functional import accuracy, f1, auroc

from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, multilabel_confusion_matrix

import seaborn as sns

from pylab import rcParams

import matplotlib.pyplot as plt

from matplotlib import rc

%matplotlib inline

%config InlineBackend.figure_format='retina'

RANDOM_SEED = 42

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

pl.seed_everything(RANDOM_SEED)

Global seed set to 42


42

In [142]:
class ToxicCommentDataModule(pl.LightningDataModule):

  def __init__(self, train_df, test_df, tokenizer, batch_size=1, max_token_len=128):

    super().__init__()

    self.batch_size = batch_size

    self.train_df = df_positive_train

    self.test_df = df_positive_test

    self.tokenizer = tokenizer

    self.max_token_len = max_token_len

  def setup(self, stage=None):

    self.train_dataset = ToxicCommentsDataset(

      self.train_df,

      self.tokenizer,

      self.max_token_len

    )

    self.test_dataset = ToxicCommentsDataset(

      self.test_df,

      self.tokenizer,

      self.max_token_len

    )

  def train_dataloader(self):

    return DataLoader(

      self.train_dataset,

      batch_size=self.batch_size,

      shuffle=True,

      num_workers=32

    )

  def val_dataloader(self):

    return DataLoader(

      self.test_dataset,

      batch_size= 1,

      num_workers=32

    )

  def test_dataloader(self):

    return DataLoader(

      self.test_dataset,

      batch_size= 1,

      num_workers=32

    )

In [143]:
N_EPOCHS = 100

MAX_TOKEN_COUNT = 512

BATCH_SIZE = 1

LEARNING_RATE = 2e-5

data_module = ToxicCommentDataModule(

  df_positive_train,

  df_positive_test,

  tokenizer,

  batch_size=BATCH_SIZE,

  max_token_len=MAX_TOKEN_COUNT

)

In [144]:
class ToxicCommentTagger(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):

    super().__init__()

    self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)

    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    self.n_training_steps = n_training_steps

    self.n_warmup_steps = n_warmup_steps

    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):

    output = self.bert(input_ids, attention_mask=attention_mask)

    output = self.classifier(output.pooler_output)

    output = torch.sigmoid(output)

    loss = 0

    if labels is not None:

        loss = self.criterion(output, labels)

    return loss, output

  def training_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]

    attention_mask = batch["attention_mask"]

    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels)

    self.log("train_loss", loss, prog_bar=True, logger=True)

    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]

    attention_mask = batch["attention_mask"]

    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels)

    self.log("val_loss", loss, prog_bar=True, logger=True)

    return loss

  def test_step(self, batch, batch_idx):

    input_ids = batch["input_ids"]

    attention_mask = batch["attention_mask"]

    labels = batch["labels"]

    loss, outputs = self(input_ids, attention_mask, labels)

    self.log("test_loss", loss, prog_bar=True, logger=True)

    return loss

  def training_epoch_end(self, outputs):

    labels = []

    predictions = []

    for output in outputs:

      for out_labels in output["labels"].detach().cpu():

        labels.append(out_labels)

      for out_predictions in output["predictions"].detach().cpu():

        predictions.append(out_predictions)

    labels = torch.stack(labels).int()

    predictions = torch.stack(predictions)

    for i, name in enumerate(LABEL_COLUMNS):

      class_roc_auc = auroc(predictions[:, i], labels[:, i])

      self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=LEARNING_RATE)

    scheduler = get_linear_schedule_with_warmup(

      optimizer,

      num_warmup_steps=self.n_warmup_steps,

      num_training_steps=self.n_training_steps

    )

    return dict(

      optimizer=optimizer,

      lr_scheduler=dict(

        scheduler=scheduler,

        interval='step'

      )

    )

In [145]:
steps_per_epoch=len(df_positive_train) // BATCH_SIZE

total_training_steps = steps_per_epoch * N_EPOCHS

warmup_steps = total_training_steps // 5

warmup_steps, total_training_steps

(25020, 125100)

In [147]:
class ToxicCommentsDataset(Dataset):

  def __init__(

    self,

    data: pd.DataFrame,

    tokenizer: BertTokenizer,

    max_token_len: int = 128

  ):

    self.tokenizer = tokenizer

    self.data = data

    self.max_token_len = max_token_len

  def __len__(self):

    return len(self.data)

  def __getitem__(self, index: int):

    data_row = self.data.iloc[index]

    comment_text = data_row.Review

    labels = data_row[LABEL_COLUMNS]

    encoding = self.tokenizer.encode_plus(

      comment_text,

      add_special_tokens=True,

      max_length=self.max_token_len,

      return_token_type_ids=False,

      padding="max_length",

      truncation=True,

      return_attention_mask=True,

      return_tensors='pt',

    )

    return dict(

      comment_text=comment_text,

      input_ids=encoding["input_ids"].flatten(),

      attention_mask=encoding["attention_mask"].flatten(),

      labels=torch.FloatTensor(labels)

    )

In [148]:
MAX_TOKEN_COUNT = 512
train_dataset = ToxicCommentsDataset(

  df_positive_train,

  tokenizer,

  max_token_len=MAX_TOKEN_COUNT

)

sample_item = train_dataset[0]

sample_item.keys()

dict_keys(['comment_text', 'input_ids', 'attention_mask', 'labels'])

In [149]:
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)

sample_batch = next(iter(DataLoader(train_dataset, batch_size=32, num_workers=2)))

sample_batch["input_ids"].shape, sample_batch["attention_mask"].shape

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(torch.Size([32, 512]), torch.Size([32, 512]))

In [150]:
model = ToxicCommentTagger(

  n_classes=len(LABEL_COLUMNS),

  n_warmup_steps=warmup_steps,

  n_training_steps=total_training_steps

)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [151]:
logger = TensorBoardLogger("lightning_logs", name="toxic-comments")
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=6)
checkpoint_callback = ModelCheckpoint(

  dirpath="checkpoints",

  filename="best-checkpoint",

  save_top_k=1,

  verbose=True,

  monitor="val_loss",

  mode="min"

)

trainer = pl.Trainer(

  logger=logger,

  checkpoint_callback=checkpoint_callback,

  callbacks=[early_stopping_callback],

  max_epochs=40,

  gpus=1,

  progress_bar_refresh_rate=30

)

/home/moritz/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:147: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f0a636b1760>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f0a636b1760>)`.
  rank_zero_deprecation(
/home/moritz/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=30)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: Tr

In [ ]:
data_module = ToxicCommentDataModule(

      df_positive_train,

      df_positive_test,

      tokenizer,

      batch_size=1,

      max_token_len=MAX_TOKEN_COUNT
    )

import time
zeitanfang = time.time()

trainer.fit(model, data_module)

zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

In [153]:
trained_model = ToxicCommentTagger.load_from_checkpoint(

  trainer.checkpoint_callback.best_model_path,

  n_classes=len(LABEL_COLUMNS)

)

trained_model = trained_model.to(device)

val_dataset = ToxicCommentsDataset(

  df_positive_test,

  tokenizer,

  max_token_len=MAX_TOKEN_COUNT

)

predictions = []

labels = []

for item in tqdm(val_dataset):
    
  _, prediction = trained_model(

    item["input_ids"].unsqueeze(dim=0).to(device),

    item["attention_mask"].unsqueeze(dim=0).to(device)

  )

  predictions.append(prediction.detach().cpu().flatten())

  labels.append(item["labels"].detach().cpu().int())

predictions = torch.stack(predictions).detach().cpu()

labels = torch.stack(labels).detach().cpu()

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/139 [00:00<?, ?it/s]

In [154]:
predictions2 = predictions.tolist()
data2={'BERT_Predictions':predictions2}
data2 = pd.DataFrame(data2)
data2.to_csv('asa/bert_predictions_pos5.csv', index=False)  
#data2.to_csv('bagging/asa/bert_predictions_pos.csv', index=False)  

In [155]:
print(data2)

                                      BERT_Predictions
0    [0.7103497385978699, 0.965736985206604, 0.9856...
1    [0.9891267418861389, 0.001976946135982871, 0.0...
2    [0.001001407508738339, 0.646804690361023, 0.00...
3    [0.002303165150806308, 0.9895614981651306, 0.0...
4    [0.9694634079933167, 0.027138778939843178, 0.0...
..                                                 ...
134  [0.0010807501384988427, 0.008470175787806511, ...
135  [0.9980494976043701, 0.0004960295627824962, 0....
136  [0.733977198600769, 0.9780758023262024, 0.0009...
137  [0.9986697435379028, 0.0009717561770230532, 0....
138  [0.9987063407897949, 0.000628592271823436, 0.0...

[139 rows x 1 columns]
